In [33]:
# for colab add this (some of them must replace)

# pip install jalali_pandas
# from google.colab import drive
# drive.mount('/content/drive')
# dollor = pd.read_excel('/content/drive/MyDrive/Marketcap Charts/dollor.xlsx')
# Mcap =  pd.read_excel('/content/drive/MyDrive/Marketcap Charts/marketcap.xlsx')

import pandas as pd
import jalali_pandas
from datetime import datetime
import plotly.express as px



dollor = pd.read_excel('./dollor.xlsx')
Mcap =  pd.read_excel('./marketcap.xlsx')

dollor.rename(columns = {'تاریخ':'DATE', 'دلار':'dollorRate'}, inplace = True)
dollor['DATE'] = dollor['DATE'].str.replace('-','/')
dollor["DATE"] =dollor["DATE"].jalali.parse_jalali("%Y/%m/%d")
dollor["DATE"] = dollor["DATE"].jalali.to_gregorian()
Mcap.drop(['ClosePrice', 'last Capital'], axis=1,inplace=True)
Mcap.rename(columns = {'market cap':'MarketCap'}, inplace = True)
Mcap = Mcap.dropna(subset=['MarketCap'])
Mcap.drop(Mcap.index[Mcap['MarketCap'] == '#VALUE!'], inplace=True)
Mcap.sort_values(by=['ShortName','DateTimeEn'], inplace=True)
Mcap.MarketCap = pd.to_numeric(Mcap.MarketCap)
Mcap['DateTimeEn'] = pd.to_datetime(Mcap['DateTimeEn']).dt.date
Mcap['DateTimeEn'] = pd.to_datetime(Mcap['DateTimeEn'], format='%Y-%m-%d')
Mcap = Mcap.query(" DateTimeEn >= '2009-03-01'" )
Mcap.reset_index(drop=True, inplace=True)
mean_dollor = dollor.groupby(pd.PeriodIndex(dollor['DATE'], freq="M"))['dollorRate'].mean().reset_index()
mean_cap = Mcap.groupby(['ShortName', pd.Grouper(key='DateTimeEn', freq='M')])['MarketCap'].mean().reset_index()
mean_cap['DateTimeEn'] = mean_cap['DateTimeEn'].dt.to_period('M')

# create a column by divide a marketcap / dollor rate
for index, row in mean_cap.iterrows():
    mean_cap.loc[index,'MarketCapindollor'] = row['MarketCap']/mean_dollor[mean_dollor['DATE'] == row['DateTimeEn']]['dollorRate'].values

#### برای رسم نمودار فقط سلول زیر را مجددا اجرا کنید





In [34]:
saham_name = input("نماد سهام مورد نظر را وارد کنید")
saham = mean_cap[mean_cap['ShortName'] == saham_name ]
saham["DateTimeEn"] = saham["DateTimeEn"].astype("datetime64[ns]")


fig = px.line(saham, x='DateTimeEn', y="MarketCapindollor")
fig.update_layout(title_text= saham_name, title_x=0.5)
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
fig.show()

/tmp/ipykernel_13153/874468782.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### پیدا کردن سهم هایی که بیشترین فاصله را از کف ۷ ساله ی خود دارند (خروجی اکسل)

In [35]:
# Assuming 'stock_data' is your original DataFrame
kaf = mean_cap
kaf['Time'] = kaf['DateTimeEn'].astype("datetime64[ns]")
start_date = pd.to_datetime('today') - pd.DateOffset(years=7)
filtered_data = kaf[kaf['Time'] >= start_date]

min_values = filtered_data.groupby('ShortName')['MarketCapindollor'].min().reset_index()

latest_values = kaf.groupby('ShortName')['MarketCapindollor'].last().reset_index()
result = pd.merge(min_values, latest_values, on='ShortName', suffixes=('_min', '_latest'))
result['Ratio'] =  result['MarketCapindollor_latest']/ result['MarketCapindollor_min']
result.sort_values(by=['Ratio'], inplace=True)

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
result.to_excel(f'{dt_string}.xlsx')